In [1]:
import os
import re
import googlemaps
import pandas as pd
from dotenv import load_dotenv, find_dotenv
# load .env
load_dotenv(find_dotenv())

True

In [2]:
# Turn off SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# Save and load csv files to scrapped
directory = "data"

# google api key
google_api_key=os.getenv('GOOGLE_API_KEY')

In [4]:
# extract number from files
def extract_numbers(filename):
    match = re.search(r'province_(\d+)_page_(\d+)\.csv', filename)
    if match:
        province = int(match.group(1))
        page = int(match.group(2))
        return (province, page)
    return (float('inf'), float('inf'))  # If not match, let it to the end

# combine all data scrapping into single csv file
def combined_data():
    df = []
    for filename in os.listdir(f'{directory}/raw'):
        if filename.endswith('.csv'):
            path_file = os.path.join(f'{directory}/raw', filename)
            df.append((path_file, extract_numbers(filename)))

    # Sort by province number and page
    df.sort(key=lambda x: x[1])

    # Stored into pandas dataframe
    sorted_df = [pd.read_csv(file[0]) for file in df]

    # Combined all pandas df
    combined_df = pd.concat(sorted_df, ignore_index=True)

    # Save to csv file with filename "siinas_data.csv" in raw data directory  
    output_file = os.path.join(f'{directory}/raw', 'siinas_data.csv')
    combined_df.to_csv(output_file, index=False)
    print("Combined data saved to raw/siinas_data.csv")

In [5]:
combined_data()

Combined data saved to raw/siinas_data.csv


In [6]:
siinas_df = pd.read_csv(f'{directory}/raw/siinas_data.csv')
siinas_df.head()

,id,name,address,phone,kbli
0,1,PR ANDRIANA MAH BENGI,"Jl.yosudarso, Blang Kolak II, Bebesen, Kabupat...",Telp. 081*********,20232.0
1,2,PR Abd Rahim,"Jl. Sp. Teritit - Pondok Baru, Uning Teritit, ...",Telp. 081*********,NaN
2,3,PR Abdul Gani M Juned,"Mutiara Baru, Mutiara Baru, Bukit, Kabupaten B...",Telp. 085*********,10794.0
3,4,PR Abdul Salam,"LR. GUNONG MEUH, Kuta Padang, Johan Pahwalan, ...",Telp. -,11040.0
4,5,PR Abdus Salam Ramli,"jl. SKB, Matang Seulimeng, Langsa Barat, Kota ...",Telp. 0,12013.0


In [11]:
# Extract province and regency information from address by extracting the address and get the last comma is province and second comma is regency
def extract_province_regency(address):
    parts = address.rsplit(',', 5)
    parts = [part.strip() for part in parts]
    province = parts[-1]
    regency = parts[-2]
    # district = f'Kecamatan {parts[-3]}'
    # sub_district = f'Kelurahan {parts[-4]}' if "Kota" in regency else f'Desa {parts[-4]}'
    # return pd.Series([province, regency, district.title(), sub_district.title()])
    return pd.Series([province, regency])

# Remove province and regency from the address
def remove_province_regency(address, province, regency):
    address = address.replace(province, "").replace(regency, "").strip()
    # Remove additional comma and space
    address = ', '.join([part.strip() for part in address.split(',') if part.strip()])
    return address

# Select only establishments from home country
def select_home_country(df):
    return df[~df['address'].str.contains('Luar Negeri')]

# Format Telp
def format_phone(phone):
    # Bersihkan spasi di awal/akhir
    phone = phone.strip()
    # Jika format tidak berisi angka yang valid, kembalikan string kosong
    if phone in ['Telp.', 'Telp. 0', 'Telp. -', ''] or '*' in phone:
        return ''
    # Hapus awalan "Telp." jika ada
    phone = re.sub(r'^Telp\.\s*', '', phone)
    # Hapus semua spasi dan tanda pemisah
    phone = re.sub(r'[^\d+]', '', phone)
    # Jika nomor diawali dengan +62, ganti dengan 0
    if phone.startswith('+62'):
        phone = '0' + phone[3:]
    return phone

# format nama perusahaan, ada yang dua spasi atau lebih
def format_name(name):
    # Mengganti spasi ganda (atau lebih) dengan satu spasi
    return re.sub(r'\s+', ' ', name).strip()

# tambah latitude dan longitude dari hasil geocoding Google Maps API
def get_coordinate(api_key, address):
    gmaps = googlemaps.Client(key=api_key)
    geocode_result = gmaps.geocode(address)
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
        return location['lat'], location['lng']
    else:
        return None, None

In [8]:
def preprocess_data(df):
  home_country_df = select_home_country(df)
  home_country_df[['province', 'regency']] = home_country_df['address'].apply(extract_province_regency)
  home_country_df['cleaned_phone'] = home_country_df['phone'].apply(format_phone)
  home_country_df['cleaned_name'] = home_country_df['name'].apply(format_name)
  home_country_df['cleaned_address'] = home_country_df.apply(lambda row: remove_province_regency(row['address'], row['province'], row['regency']), axis=1)
  home_country_df['kbli'] = home_country_df['kbli'].apply(lambda row: str(int(row)) if not pd.isna(row) else '')
  home_country_df['id'] = range(1, len(home_country_df) + 1)
  return home_country_df

In [ ]:
final_df = preprocess_data(siinas_df)

In [12]:
# Mendapatkan koordinat dari perusahaan berdasarkan alamat
# initial
final_df['latitude'] = None
final_df['longitude'] = None

# Looping
for index, row in final_df.iterrows():
    address = row['address']  #ganti kolom yang sesuai dengan penamaan data CSV
    lat, lng = get_coordinate(google_api_key, address)
    final_df.at[index, 'latitude'] = lat
    final_df.at[index, 'longitude'] = lng

# cek final_df
final_df.head()

,id,name,address,phone,kbli,province,regency,cleaned_phone,cleaned_name,cleaned_address,Latitude,Longitude,latitude,longitude
0,1,PR ANDRIANA MAH BENGI,"Jl.yosudarso, Blang Kolak II, Bebesen, Kabupat...",Telp. 081*********,20232,Nangroe Aceh Darussalam,Kabupaten Aceh Tengah,,PR ANDRIANA MAH BENGI,"Jl.yosudarso, Blang Kolak II, Bebesen",None,None,4.622940,96.837448
1,2,PR Abd Rahim,"Jl. Sp. Teritit - Pondok Baru, Uning Teritit, ...",Telp. 081*********,,Nangroe Aceh Darussalam,Kabupaten Bener Meriah,,PR Abd Rahim,"Jl. Sp. Teritit - Pondok Baru, Uning Teritit, ...",None,None,4.759029,96.916888
2,3,PR Abdul Gani M Juned,"Mutiara Baru, Mutiara Baru, Bukit, Kabupaten B...",Telp. 085*********,10794,Nangroe Aceh Darussalam,Kabupaten Bener Meriah,,PR Abdul Gani M Juned,"Mutiara Baru, Mutiara Baru, Bukit",None,None,4.675053,96.833760
3,4,PR Abdul Salam,"LR. GUNONG MEUH, Kuta Padang, Johan Pahwalan, ...",Telp. -,11040,Nangroe Aceh Darussalam,Kabupaten Aceh Barat,,PR Abdul Salam,"LR. GUNONG MEUH, Kuta Padang, Johan Pahwalan",None,None,4.149671,96.121665
4,5,PR Abdus Salam Ramli,"jl. SKB, Matang Seulimeng, Langsa Barat, Kota ...",Telp. 0,12013,Nangroe Aceh Darussalam,Kota Langsa,,PR Abdus Salam Ramli,"jl. SKB, Matang Seulimeng, Langsa Barat",None,None,4.484066,97.971547


In [ ]:
# Save to csv file with filename "siinas_data.csv" in raw data directory  
output_file = os.path.join(f'{directory}/processed', 'siinas_data.csv')
final_df.to_csv(output_file, index=False)
print("Combined data saved to processed/siinas_data.csv")

In [ ]:
# Save to csv file
final_file = os.path.join(f'../../datasets/establishments', 'siinas_data.csv')
final_df.rename(
  columns={
    'cleaned_address': 'alamat', 
    'cleaned_phone': 'telepon', 
    'cleaned_name': 'nama',
    'province': 'provinsi',
    'regency': 'kabupaten_kota',
    'kbli': 'kelompok_kbli'
    }
    , inplace=True)
final_df[['id', 'nama', 'alamat', 'latitude', 'longitude', 'telepon']].to_csv(final_file, index=False)
print("Final data saved to datasets/siinas_data.csv")